In [ ]:
import MySQLdb
from pandas.io import sql
import pandas as pd
import numpy as np
conn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="openmrs") 
openmrscursor = conn.cursor() 

In [ ]:
df_concept = pd.read_sql('select  * from concept where class_id = 16', con = conn)
df_concept

In [ ]:
mimicconn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="mimic4") 
mimiccursor = mimicconn.cursor()

In [ ]:
concept_freq = pd.read_sql('select distinct frequency from pharmacy', con = mimicconn) 
concept_freq

In [ ]:
concept_freq['frequency'] = concept_freq['frequency'].str.lower()
concept_freq = concept_freq.drop_duplicates()

In [ ]:
concept_freq = concept_freq.rename(columns={'frequency':'description'})

concept_freq['date_created']= '2022-01-01'
import uuid
concept_freq['uuid'] = [uuid.uuid4() for x in range(len(concept_freq))]
concept_freq['creator'] = 1
concept_freq['datatype_id'] = 4
concept_freq['class_id'] = 16

In [ ]:
concept_freq.dropna(inplace = True)

In [ ]:
column_names = ['description', 'datatype_id', 'class_id', 'creator', 'date_created', 'uuid']
concept_freq = concept_freq.reindex(columns = column_names)
concept_freq.isnull().sum()

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in concept_freq.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in concept_freq.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `concept` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 